In [2]:
# !pip install Keras
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.applications import xception
# from keras.models import Sequential, Model, load_model
# from keras.layers import Conv2D, MaxPooling2D
# from keras.layers import Activation, Dropout, Flatten, Dense
# from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
# from keras import regularizers
# from keras import backend as K
# import sys
# from matplotlib import pyplot
# from keras.layers import Conv2D
# from keras.layers import MaxPooling2D
# from keras.optimizers import SGD
# from keras.utils import multi_gpu_model
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# from keras.callbacks import ModelCheckpoint
# import pandas as pd
# from keras.layers.normalization import BatchNormalization
# from tensorflow.keras.callbacks import TensorBoard
# import time

You should consider upgrading via the 'pip install --upgrade pip' command.


Using TensorFlow backend.


In [1]:
!pip install Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import xception
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
import sys
from matplotlib import pyplot
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.optimizers import SGD
from keras.utils import multi_gpu_model
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
from keras.callbacks import ModelCheckpoint
import pandas as pd
from keras.layers.normalization import BatchNormalization
# from tensorflow.keras.callbacks import TensorBoard
import time
import keras

     |████████████████████████████████| 378kB 10.4MB/s eta 0:00:01
     |████████████████████████████████| 276kB 5.1MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/e4/76/4d/a95b8dd7b452b69e8ed4f68b69e1b55e12c9c9624dd962b191
Successfully built pyyaml
You should consider upgrading via the 'pip install --upgrade pip' command.


Using TensorFlow backend.


In [2]:
# import tensorflow as tf
# import keras.backend.tensorflow_backend as ktf


# def get_session(gpu_fraction=0.6):
#     gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction,
#                                 allow_growth=True)
#     return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


# ktf.set_session(get_session())

In [2]:
K.set_learning_phase(1)




In [3]:
img_width, img_height = 299, 299
nb_train_samples = 539231
nb_validation_samples = 240424
epochs = 10
batch_size = 100
n_classes = 1

In [4]:
train_data_dir = "../../../../../raid/Data/Master_Dataset/mix/train"
validation_data_dir = "../../../../../raid/Data/Master_Dataset/mix/validation"

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode = 'constant',
    cval = 1,
    rotation_range = 5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 539231 images belonging to 2 classes.
Found 240424 images belonging to 2 classes.


In [ ]:
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# config.log_device_placement = True  # to log device placement (on which device the operation ran)
#                                     # (nothing gets printed in Jupyter, only if you run it standalone)
# sess = tf.Session(config=config)
# set_session(sess)  #

In [5]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess) 

In [6]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto',min_delta=1e-2, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=1, mode='auto',verbose=1, min_delta=1e-2, min_lr=0.00001)

In [7]:
i = 0
class LossHistory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        global i
        self.model.save("paper/weights_of_epoch_" + str(i) + ".hdf5")
#         summarize_diagnostics(self.model_history, i)
        i += 1
        

In [8]:
# final model train 
n_classes = 1
base_model = xception.Xception(input_shape=(img_width, img_height, 3),
                                     weights="xception_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                     include_top=False,
                                     pooling='avg')


for layer in base_model.layers:
    layer.trainable = True
x = base_model.output
x = Dense(n_classes, activation='sigmoid', name = 'prediction')(x)
model = Model(inputs = base_model.input, outputs = x)
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
____________________

In [9]:
e = 10
# model = tf.keras.models.load_model('/home/vision/mrunal/detect_deepfake/densenet121/paper/weights_of_epoch_0.hdf5')
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model = multi_gpu_model(model, gpus=2)
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
final_location = []
lh = LossHistory()
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [lh, early_stop, reduce_lr]

model_history = model.fit_generator(
train_generator,
epochs=e,
steps_per_epoch = nb_train_samples // train_generator.batch_size,
validation_data=validation_generator,
verbose=1,
validation_steps=nb_validation_samples // validation_generator.batch_size,
callbacks=callbacks_list)

#model.save("weights_best_" + str(e) + ".hdf5")

# _,acc=model.evaluate_generator(validation_generator, steps=nb_validation_samples, max_queue_size=10,verbose=1, workers=1, use_multiprocessing=False)
# print('> %.3f' % (acc * 100.0))

loss=model_history.history['loss']
acc=model_history.history['accuracy']
valacc=model_history.history['val_accuracy']
valloss=model_history.history['val_loss']
location = [e,loss,acc,valacc, valloss]
final_location.append(location)
save = pd.DataFrame(final_location,columns=['epochs','loss','acc','valacc','valloss'])
save.to_csv('/home/vision/mrunal/detect_deepfake/xception/paper/xception_log_'+str(e)+'.csv')

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
5392/5392 [==============================] - 10609s 2s/step - loss: 0.1961 - accuracy: 0.9137 - val_loss: 0.2147 - val_accuracy: 0.8782
Epoch 2/10
5392/5392 [==============================] - 10458s 2s/step - loss: 0.0711 - accuracy: 0.9722 - val_loss: 0.3734 - val_accuracy: 0.8799

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 3/10
5392/5392 [==============================] - 10979s 2s/step - loss: 0.0519 - accuracy: 0.9795 - val_loss: 0.3235 - val_accuracy: 0.8809

Epoch 00003: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 4/10
5392/5392 [==============================] - 9784s 2s/step - loss: 0.0489 - accuracy: 0.9808 - val_loss: 0.2942 - val_accuracy: 0.8786
Restoring model weights from the end of the best epoch

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 00004: early stopping


In [12]:
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [1]:
!nvidia-smi

Thu Jan 30 08:31:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:89:00.0 Off |                    0 |
| N/A   56C    P0   266W / 300W |  14377MiB / 32480MiB |     77%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   39C    P0    60W / 300W |  14053MiB / 32480MiB |     19%      Default |
+-------------------------------+----------------------+----------------------+
        

In [13]:
y = 1
base_model = xception.Xception(input_shape=(img_width, img_height, 3),
                                     weights="xception_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                     include_top=False,
                                     pooling='avg')


for layer in base_model.layers:
    layer.trainable = True


num_classes = 1
e=4

new_model = Model(inputs = base_model.input, outputs = base_model.layers[-2].output)
x = new_model.output

model = Sequential()

dense_layers = [1, 2, 3]
layer_sizes = [32, 64, 128]
conv_layers = [ 1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)
            
            final_location=[]
            location=[]
            
            
            model.add(Conv2D(layer_size, (3, 3), input_shape = (img_width, img_height, 3), activation = 'relu'))

            model.add(MaxPooling2D(pool_size=(2, 2)))
            
            
            for l in range(conv_layer-1):
                
                
                model.add(Conv2D(layer_size, (3, 3), activation = 'relu'))

                model.add(MaxPooling2D(pool_size=(2, 2)))

            
                
            model.add(Flatten())
            
                

            for _ in range(dense_layer):
            
                model.add(Dense(layer_size, activation = 'relu'))


            model.add(Dense(1, activation = 'sigmoid'))

            

            
            model.summary()
            
            sgd = SGD(lr=0.001, decay=1e-5, momentum=0.9, nesterov=True)
    
            model = multi_gpu_model(model, gpus=2)
        
            model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

            filepath="final_mix_weights_best_" + str(y)+ '_'+ str(e) + ".hdf5"
            
            checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
            
            callbacks_list = [checkpoint,early_stop, reduce_lr]

            model_history = model.fit_generator(
            train_generator,
            epochs=e,
            steps_per_epoch=nb_train_samples // batch_size,
            validation_data=validation_generator,
            verbose=1,
            validation_steps=nb_validation_samples // batch_size,
            callbacks=callbacks_list)
            
            y = y + 1
            
            summarize_diagnostics(model_history)
            loss=model_history.history['loss']
            acc=model_history.history['accuracy']
            valacc=model_history.history['val_accuracy']
            valloss=model_history.history['val_loss']
            location = [e,loss,acc,valacc, valloss]
            final_location.append(location)
            save = pd.DataFrame(final_location,columns=['epochs','loss','acc','valacc','valloss'])
            save.to_csv('final_mix_xception_log_'+str(e)+'.csv')

1-conv-32-nodes-1-dense-1579241103
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 297, 297, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 148, 148, 32)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 700928)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                22429728  
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 22,430,657
Trainable params: 22,430,657
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/4
2794/3594 [====

KeyboardInterrupt: 

In [22]:
# base_model = xception.Xception(input_shape=(img_width, img_height, 3),
#                                      weights="xception_weights_tf_dim_ordering_tf_kernels_notop.h5",
#                                      include_top=False,
#                                      pooling='avg')


# for layer in base_model.layers:
#     layer.trainable = True


num_classes = 1
e=4

# new_model = Model(inputs = base_model.input, outputs = base_model.layers[-2].output)
# x = new_model.output




dense_layers = [1, 2, 3]
layer_sizes = [32, 64, 128]
conv_layers = [ 1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)
            
            final_location=[]
            location=[]
            
            model = Sequential()
            
            model.add(Conv2D(layer_size, (3, 3), input_shape = (img_width, img_height, 3), activation = 'relu'))

            model.add(MaxPooling2D(pool_size=(2, 2)))
            
            
            for l in range(conv_layer-1):
                
                
                model.add(Conv2D(layer_size, (3, 3), activation = 'relu'))

                model.add(MaxPooling2D(pool_size=(2, 2)))

            
                
            model.add(Flatten())
            
                

            for _ in range(dense_layer):
            
                model.add(Dense(layer_size, activation = 'relu'))


            model.add(Dense(1, activation = 'sigmoid'))

            

            
            model.summary()
            

            

1-conv-32-nodes-1-dense-1577965801
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 297, 297, 32)      896       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 148, 148, 32)      0         
_________________________________________________________________
flatten_29 (Flatten)         (None, 700928)            0         
_________________________________________________________________
dense_32 (Dense)             (None, 32)                22429728  
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 33        
Total params: 22,430,657
Trainable params: 22,430,657
Non-trainable params: 0
_________________________________________________________________
2-conv-32-nodes-1-dense-1577965801
Model: "sequential_10"
_______________

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_60 (Conv2D)           (None, 297, 297, 128)     3584      
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 148, 148, 128)     0         
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 146, 146, 128)     147584    
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 73, 73, 128)       0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 71, 71, 128)       147584    
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
flatten_37 (Flatten)         (None, 156800)          

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_72 (Conv2D)           (None, 297, 297, 64)      1792      
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 148, 148, 64)      0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 146, 146, 64)      36928     
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 71, 71, 64)        36928     
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
flatten_43 (Flatten)         (None, 78400)           

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_84 (Conv2D)           (None, 297, 297, 32)      896       
_________________________________________________________________
max_pooling2d_65 (MaxPooling (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 146, 146, 32)      9248      
_________________________________________________________________
max_pooling2d_66 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 71, 71, 32)        9248      
_________________________________________________________________
max_pooling2d_67 (MaxPooling (None, 35, 35, 32)        0         
_________________________________________________________________
flatten_49 (Flatten)         (None, 39200)           

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_96 (Conv2D)           (None, 297, 297, 128)     3584      
_________________________________________________________________
max_pooling2d_77 (MaxPooling (None, 148, 148, 128)     0         
_________________________________________________________________
conv2d_97 (Conv2D)           (None, 146, 146, 128)     147584    
_________________________________________________________________
max_pooling2d_78 (MaxPooling (None, 73, 73, 128)       0         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 71, 71, 128)       147584    
_________________________________________________________________
max_pooling2d_79 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
flatten_55 (Flatten)         (None, 156800)          

In [9]:
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [6]:
def define_model():
    base_model = xception.Xception(input_shape=(img_width, img_height, 3),
                                     weights="xception_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                     include_top=False,
                                     pooling='avg')
   
    for layer in base_model.layers:
        layer.trainable = True
    
    new_model = Model(base_model.inputs, base_model.layers[-2].output)
    
    model=new_model.output
    model = Conv2D(64, (3, 3), activation = 'relu')(model)
    model = MaxPooling2D(pool_size = (2, 2))(model)
    model = Dropout(0.50)(model)
    
    model = Flatten()(model)
    model = Dense(512,activation='relu')(model)
    model = Dropout(0.5)(model)
    model = Dense(1, activation='sigmoid')(model)
    predictions = Dense(n_classes, activation='sigmoid')(model)
    
    model = Model(inputs=new_model.input, outputs=predictions)
    
    return model

In [ ]:
model = Sequential()
model = Conv2D(input_shape=(224,224,3), data_format = 'channels_last', )

In [7]:
model=define_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
____________________

In [7]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto',min_delta=1e-2, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, mode='auto',verbose=1, min_delta=1e-2, min_lr=0.00001)


In [8]:
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [10]:
epochs=[6,10,14]
x=1
for e in epochs:
    final_location=[]
    location=[]
    #base_model=Sequential()
    model=define_model()
    
    sgd = SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True)
    model = multi_gpu_model(model, gpus=1)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    
    filepath="weights_best_" + str(e) + ".hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint,early_stop, reduce_lr]
    
    model_history = model.fit_generator(
    train_generator,
    epochs=e,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    verbose=1,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks_list)
    
    #model.save("weights_best_" + str(e) + ".hdf5")
    
    _,acc=model.evaluate_generator(validation_generator, steps=nb_validation_samples, max_queue_size=10,verbose=1, workers=1, use_multiprocessing=False)
    print('> %.3f' % (acc * 100.0))
    summarize_diagnostics(model_history)
    
    loss=model_history.history['loss']
    acc=model_history.history['accuracy']
    valacc=model_history.history['val_accuracy']
    valloss=model_history.history['val_loss']
    location = [e,loss,acc,valacc, valloss]
    final_location.append(location)
    save = pd.DataFrame(final_location,columns=['epochs','loss','acc','valacc','valloss'])
    save.to_csv('xception_log_'+str(e)+'.csv')
    

Instructions for updating:
Use tf.cast instead.
Epoch 1/6


ResourceExhaustedError: OOM when allocating tensor with shape[150,128,147,147] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block2_sepconv2_bn_1/FusedBatchNormV3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node Mean}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [10]:
model = keras.models.load_model('paper/weights_of_epoch_2.hdf5')
model.summary()

In [11]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            20863529    lambda_1[0][0]                   
                                                                 lambda_2[0][0]             